<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/autogluon/experiments/covtype_table_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install scikit-learn==0.23.1
%pip install --upgrade mxnet-cu101
%pip install autogluon

  Using cached https://files.pythonhosted.org/packages/d9/3a/eb8d7bbe28f4787d140bb9df685b7d5bf6115c0e2a969def4027144e98b6/scikit_learn-0.23.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: autogluon 0.0.12 has requirement scikit-learn<0.23,>=0.22.0, but you'll have scikit-learn 0.23.1 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
Requirement already up-to-date: mxnet-cu101 in /usr/local/lib/python3.6/dist-packages (1.6.0)
  Using cached https://files.pythonhosted.org/packages/5e/d8/312e03adf4c78663e17d802fe2440072376fee46cada1404f1727ed77a32/scikit_learn-0.22.2.post1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
%cd /gdrive/My Drive/機械学習練習/AutoML/AutoGluon/experiments
%ls

/gdrive/My Drive/機械学習練習/AutoML/AutoGluon/experiments
AutogluonModels/         dask-worker-space/  models/
covtype_table_cls.ipynb  dataset/            sample_table_cls.ipynb


# Predicting Columns in a Table - In Depth
テーブルデータを使った予測の練習[https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-indepth.html]([https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-indepth.html)

In [ ]:
import datetime, os

import numpy as np
import pandas as pd

import autogluon as ag
from autogluon import TabularPrediction as task
from autogluon.task.tabular_prediction import TabularDataset
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split

## データの読み込み

In [ ]:
SEED = 2

fetch_data = fetch_covtype(random_state=SEED)

train_X, test_X, train_y, test_y = train_test_split(fetch_data.data, fetch_data.target-1,
                                                    test_size=0.2, random_state=SEED)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  test_size=0.2, random_state=SEED)
print('train_X:{}, train_y:{}'.format(train_X.shape, train_y.shape))
print('val_X:{}, val_y:{}'.format(val_X.shape, val_y.shape))
print('test_X:{}, test_y:{}'.format(test_X.shape, test_y.shape))

label_col = 'class'
f_cols = []
for i in range(train_X.shape[1]):
  f_cols.append('f_'+str(i+1))

train_X:(371847, 54), train_y:(371847,)
val_X:(92962, 54), val_y:(92962,)
test_X:(116203, 54), test_y:(116203,)


### モデルの入力形式に合わせたデータの変型

In [ ]:
def convert_to_TabularDataset(X, y, f_cols, y_col):
  df = pd.DataFrame(data=X,columns=f_cols)
  df.insert(loc=0, column=y_col, value=y)
  return TabularDataset(df = df)

In [ ]:
train_td = convert_to_TabularDataset(train_X, train_y, f_cols, label_col)
val_td = convert_to_TabularDataset(val_X, val_y, f_cols, label_col)
test_td = convert_to_TabularDataset(test_X, test_y, f_cols, label_col)
print('データの型:{}'.format(type(train_td)))

データの型:<class 'autogluon.task.tabular_prediction.dataset.TabularDataset'>


## ベストなモデルを探索する

### 実験の詳細を指定

In [ ]:
# 実験の最大許容時間の設定
time_limits = 300  #@param {type: "number"}

# 訓練後のモデル出力先
output_directory = 'models'
model_name = 'table_cls' + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
output_model_path = os.path.join(output_directory, model_name)

In [ ]:
predictor = task.fit(train_data=train_td, tuning_data=val_td, label=label_col,
                     auto_stack=True, # アンサンブルの戦略も自動で行ってほしい場合
                     #num_bagging_folds=5, stack_ensemble_levels=1, #決め打ちしたい場合
                     time_limits=time_limits)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200725_143521/
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to AutogluonModels/ag-20200725_143521/
AutoGluon Version:  0.0.12
Train Data Rows:    371847
Train Data Columns: 55
Tuning Data Rows:    92962
Tuning Data Columns: 55
Preprocessing data ...
Here are the 7 unique label values in your data:  [1, 2, 0, 5, 6, 4, 3]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 7
Feature Generator processed 464809 data points with 54 features
Original Features (raw dtypes):
	float64 features: 54
Original Features (inferred dtypes):
	float features: 54
Generated Features (special dtypes):
Final Features (raw dtypes):
	float fea

### 結果の確認

In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0  LightGBMClassifier_STACKER_l1   0.794234       4.798083  157.103575                1.756800          71.453314            1       True
1        weighted_ensemble_k0_l2   0.794234       4.871034  157.200658                0.072951           0.097083            2       True
2  LightGBMClassifier_STACKER_l0   0.771814       3.041282   85.650261                3.041282          85.650261            0       True
3        weighted_ensemble_k0_l1   0.771814       3.112620   85.752110                0.071338           0.101849            1       True
Number of models trained: 4
Types of models trained:
{'WeightedEnsembleModel', 'StackerEnsembleModel_LGB'}
Bagging used: True  (with 10 folds)
Stack-ensembling used: True  (with 1 levels)
Hyperparameter-tuning used: False 
User-specified hyperparam